In [2]:
from coffea.nanoevents import NanoEventsFactory, BaseSchema, PFNanoAODSchema
import json
import fastjet
import numpy as np
import awkward as ak
from coffea import processor
import hist
import coffea.nanoevents.methods.vector as vector
import warnings
import hist.dask as dhist
import dask
import pickle
import os
#import distributed
from ndcctools.taskvine import DaskVine
import time

In [3]:
ak.__version__

'2.5.2'

In [4]:
warnings.filterwarnings("ignore", "Found duplicate branch")
warnings.filterwarnings("ignore", "Missing cross-reference index for")
warnings.filterwarnings("ignore", "dcut")
warnings.filterwarnings("ignore", "Please ensure")
warnings.filterwarnings("ignore", "The necessary")

In [5]:
hbb_files = os.listdir('/project01/ndcms/cmoore24/signal/hbb')

In [6]:
len(hbb_files)

25

In [7]:
hbb_files = hbb_files[1:3]

In [9]:
hbb = NanoEventsFactory.from_root(
    {'/project01/ndcms/cmoore24/signal/hbb/' + hbb_files[0]: "/Events"},
    #permit_dask=True,
    schemaclass=PFNanoAODSchema,
    metadata={"dataset": "QCD_Pt_1000to1400"},
).events()

In [11]:
class MyProcessor_Background(processor.ProcessorABC):
        
        def __init__(self):
            pass
    
        
        def process(self, events):
            dataset = events.metadata['dataset']
            
            fatjet = events.FatJet 
            
            cut = ((fatjet.pt > 300) & (fatjet.msoftdrop > 110) & 
                   (fatjet.msoftdrop < 140) & (abs(fatjet.eta) < 2.5))  #& (fatjet.btagDDBvLV2 < 0.50)
            
            boosted_fatjet = fatjet[cut]
            
            cmssw_n2 = (
                dhist.Hist.new
                .Reg(40, 0, 0.5, name='cmssw_n2', label='CMSSW_N2')
                .Weight()
            )

            cmssw_n2.fill(cmssw_n2=ak.flatten(boosted_fatjet.n2b1))
            
            return {
                dataset: {
                    "entries": ak.count(events.event, axis=None),
                    "N2": cmssw_n2,
                }
            }
        
        def postprocess(self, accumulator):
            pass

In [12]:
result = MyProcessor_Background().process(hbb)

In [13]:
dask.compute(result)

({'QCD_Pt_1000to1400': {'entries': 6000,
   'N2': Hist(Regular(40, 0, 0.5, name='cmssw_n2', label='CMSSW_N2'), storage=Weight()) # Sum: WeightedSum(value=637, variance=637)}},)

In [12]:
type(ak.firsts(q1418.FatJet))#.n2b1)

coffea.nanoevents.methods.nanoaod.FatJetArray

In [9]:
q1418.FatJet.n2b1

<Array [[0.385], [...], ..., [0.282, -1e+05]] type='6000 * var * float32[pa...'>